# 📊 **Exercise 7: Tissue Maturation Monitoring Dashboard**

---

## 🎯 **Learning Objectives**

By completing this exercise, you will be able to:

1. **Integrate multiple maturation parameters** into a unified monitoring system
2. **Analyze time-course data** for tissue development assessment
3. **Calculate composite maturation scores** from multiple biomarkers
4. **Compare culture conditions** using multivariate analysis
5. **Identify optimization opportunities** through parameter correlation
6. **Make data-driven decisions** for culture protocol refinement

---

## 🔬 **Real-World Context**

Successful tissue engineering requires **comprehensive monitoring** of multiple parameters:

**📈 Cell Parameters:**
- Viability, proliferation, metabolic activity

**🧬 ECM Production:**
- Collagen deposition, GAG content, elastin

**💪 Mechanical Properties:**
- Stiffness (Young's modulus), tensile strength, elasticity

**🔄 Metabolic Markers:**
- Glucose consumption, lactate production, oxygen uptake

**🧪 Gene Expression:**
- Tissue-specific markers, differentiation markers

**🎯 Challenge:** How do you integrate all these parameters into a **single, actionable assessment** of tissue maturation?

---

## 📚 **Theoretical Background**

### **Tissue Maturation Score (TMS)**

A composite metric that integrates multiple parameters:

$$\text{TMS} = \sum_{i=1}^{n} w_i \cdot \frac{P_i - P_{i,\text{min}}}{P_{i,\text{target}} - P_{i,\text{min}}}$$

Where:
- $P_i$ = Current value of parameter $i$
- $P_{i,\text{min}}$ = Baseline value (e.g., day 0)
- $P_{i,\text{target}}$ = Target value (e.g., native tissue)
- $w_i$ = Weighting factor for parameter importance
- $\sum w_i = 1$ (weights sum to 100%)

**TMS = 0%:** Immature tissue (baseline)  
**TMS = 100%:** Fully mature tissue (native-like)

---

### **Key Maturation Parameters**

**1. Cell Viability & Proliferation**
$$\text{Viability} = \frac{\text{Live Cells}}{\text{Total Cells}} \times 100\%$$

$$\text{Growth Rate} = \frac{\ln(N_t) - \ln(N_0)}{t} \quad [\text{day}^{-1}]$$

**2. ECM Content**
- **Collagen:** Primary structural protein (mg/construct)
- **GAG (Glycosaminoglycans):** Hydration and compressive resistance (µg/construct)
- **Elastin:** Tissue elasticity (mg/construct)

**3. Mechanical Properties**
$$E = \frac{\sigma}{\varepsilon} \quad [\text{kPa}]$$

Where $E$ = Young's modulus, $\sigma$ = stress, $\varepsilon$ = strain

**4. Metabolic Activity**
$$\text{Glucose Consumption Rate} = \frac{C_{\text{in}} - C_{\text{out}}}{t \cdot N_{\text{cells}}} \quad [\text{pmol/cell/h}]$$

$$\text{Lactate Yield} = \frac{\text{Lactate Produced}}{\text{Glucose Consumed}}$$

**5. Gene Expression**
$$\text{Fold Change} = 2^{-\Delta\Delta C_t}$$

---

## 💻 **Let's Start Coding!**

Run the code cells below to import libraries and set up the interactive dashboard.

In [ ]:
# Import required libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Circle
import pandas as pd
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import display, HTML

# Configure plotting style
plt.style.use('seaborn-v0_8-darkgrid')
plt.rcParams['figure.figsize'] = (14, 10)
plt.rcParams['font.size'] = 11

print("✅ Libraries imported successfully!")
print("📊 Ready to create your Tissue Maturation Dashboard!")

---

## 🗂️ **SECTION 1: Define Tissue Maturation Database**

First, let's define **baseline**, **target**, and **current** values for different tissue types.

In [ ]:
# ============================================================================
# TISSUE MATURATION DATABASE
# ============================================================================

# Define maturation parameters for different tissue types
TISSUE_DATABASE = {
    'Cartilage': {
        'parameters': {
            'Cell_Viability': {'baseline': 95, 'target': 90, 'unit': '%', 'weight': 0.10},
            'Cell_Density': {'baseline': 5e6, 'target': 60e6, 'unit': 'cells/mL', 'weight': 0.10},
            'Collagen_Content': {'baseline': 2, 'target': 150, 'unit': 'mg/g', 'weight': 0.20},
            'GAG_Content': {'baseline': 5, 'target': 60, 'unit': 'mg/g', 'weight': 0.25},
            'Youngs_Modulus': {'baseline': 10, 'target': 800, 'unit': 'kPa', 'weight': 0.20},
            'COL2_Expression': {'baseline': 1.0, 'target': 15.0, 'unit': 'fold', 'weight': 0.10},
            'ACAN_Expression': {'baseline': 1.0, 'target': 20.0, 'unit': 'fold', 'weight': 0.05}
        },
        'description': 'Engineered articular cartilage'
    },
    
    'Bone': {
        'parameters': {
            'Cell_Viability': {'baseline': 95, 'target': 85, 'unit': '%', 'weight': 0.10},
            'Cell_Density': {'baseline': 5e6, 'target': 30e6, 'unit': 'cells/mL', 'weight': 0.10},
            'Collagen_Content': {'baseline': 5, 'target': 200, 'unit': 'mg/g', 'weight': 0.15},
            'Calcium_Content': {'baseline': 2, 'target': 250, 'unit': 'mg/g', 'weight': 0.30},
            'Youngs_Modulus': {'baseline': 50, 'target': 15000, 'unit': 'kPa', 'weight': 0.20},
            'ALP_Activity': {'baseline': 10, 'target': 200, 'unit': 'U/L', 'weight': 0.10},
            'RUNX2_Expression': {'baseline': 1.0, 'target': 25.0, 'unit': 'fold', 'weight': 0.05}
        },
        'description': 'Engineered bone tissue'
    },
    
    'Cardiac': {
        'parameters': {
            'Cell_Viability': {'baseline': 90, 'target': 85, 'unit': '%', 'weight': 0.15},
            'Cell_Density': {'baseline': 10e6, 'target': 100e6, 'unit': 'cells/mL', 'weight': 0.10},
            'Collagen_Content': {'baseline': 3, 'target': 40, 'unit': 'mg/g', 'weight': 0.10},
            'Contraction_Force': {'baseline': 0.1, 'target': 5.0, 'unit': 'mN', 'weight': 0.25},
            'Beat_Rate': {'baseline': 0, 'target': 70, 'unit': 'bpm', 'weight': 0.15},
            'Troponin_Expression': {'baseline': 1.0, 'target': 30.0, 'unit': 'fold', 'weight': 0.15},
            'Connexin43': {'baseline': 1.0, 'target': 12.0, 'unit': 'fold', 'weight': 0.10}
        },
        'description': 'Engineered cardiac tissue'
    },
    
    'Vascular': {
        'parameters': {
            'Cell_Viability': {'baseline': 95, 'target': 90, 'unit': '%', 'weight': 0.10},
            'Cell_Density': {'baseline': 8e6, 'target': 50e6, 'unit': 'cells/mL', 'weight': 0.10},
            'Collagen_Content': {'baseline': 4, 'target': 100, 'unit': 'mg/g', 'weight': 0.15},
            'Elastin_Content': {'baseline': 1, 'target': 80, 'unit': 'mg/g', 'weight': 0.20},
            'Burst_Pressure': {'baseline': 20, 'target': 2000, 'unit': 'mmHg', 'weight': 0.25},
            'Compliance': {'baseline': 1, 'target': 8, 'unit': '%/100mmHg', 'weight': 0.15},
            'eNOS_Expression': {'baseline': 1.0, 'target': 10.0, 'unit': 'fold', 'weight': 0.05}
        },
        'description': 'Engineered blood vessel'
    }
}

print("✅ Tissue database loaded successfully!")
print(f"\n📁 Available tissue types: {', '.join(TISSUE_DATABASE.keys())}")
print("\n💡 Each tissue has 7 key maturation parameters with baseline, target, and weighting values.")

---

## 📈 **SECTION 2: Time-Course Maturation Data**

Let's simulate realistic tissue maturation over time with different culture conditions.

In [ ]:
def generate_maturation_data(tissue_type, culture_condition, time_days):
    """
    Generate realistic time-course maturation data
    
    Culture conditions affect maturation kinetics:
    - Optimal: Fast maturation, high final values
    - Standard: Moderate maturation
    - Suboptimal: Slow maturation, lower final values
    """
    
    params = TISSUE_DATABASE[tissue_type]['parameters']
    time_points = np.linspace(0, time_days, 20)
    
    # Define culture condition parameters
    conditions = {
        'Optimal': {'rate': 0.15, 'plateau': 0.95, 'noise': 0.03},
        'Standard': {'rate': 0.10, 'plateau': 0.80, 'noise': 0.05},
        'Suboptimal': {'rate': 0.06, 'plateau': 0.60, 'noise': 0.08}
    }
    
    cond = conditions[culture_condition]
    
    maturation_data = {}
    
    for param_name, param_info in params.items():
        baseline = param_info['baseline']
        target = param_info['target']
        
        # Sigmoidal growth curve (logistic function)
        progress = cond['plateau'] / (1 + np.exp(-cond['rate'] * (time_points - time_days/2)))
        
        # Add biological noise
        noise = np.random.normal(0, cond['noise'], len(time_points))
        progress = np.clip(progress + noise, 0, 1)
        
        # Calculate actual values
        values = baseline + progress * (target - baseline)
        
        # Special handling for viability (can decrease)
        if param_name == 'Cell_Viability':
            decline_rate = {'Optimal': 0.02, 'Standard': 0.05, 'Suboptimal': 0.10}
            values = baseline - decline_rate[culture_condition] * time_points + noise * 2
            values = np.clip(values, target - 10, baseline)
        
        maturation_data[param_name] = values
    
    maturation_data['Time'] = time_points
    
    return pd.DataFrame(maturation_data)

print("✅ Time-course data generator ready!")
print("\n📊 This function simulates tissue maturation over time with:")
print("   • Sigmoidal growth kinetics")
print("   • Culture condition effects")
print("   • Biological variability (noise)")

---

## 🎯 **SECTION 3: Calculate Tissue Maturation Score (TMS)**

In [ ]:
def calculate_TMS(tissue_type, current_values):
    """
    Calculate Tissue Maturation Score (TMS)
    
    TMS = Σ [weight_i × (current_i - baseline_i)/(target_i - baseline_i)]
    
    Returns: TMS (0-100%) and individual parameter scores
    """
    
    params = TISSUE_DATABASE[tissue_type]['parameters']
    
    parameter_scores = {}
    weighted_scores = []
    
    for param_name, param_info in params.items():
        baseline = param_info['baseline']
        target = param_info['target']
        weight = param_info['weight']
        current = current_values.get(param_name, baseline)
        
        # Calculate normalized score (0-1)
        if target != baseline:
            score = (current - baseline) / (target - baseline)
        else:
            score = 1.0
        
        # Clip to 0-1 range
        score = np.clip(score, 0, 1)
        
        parameter_scores[param_name] = score * 100  # Convert to percentage
        weighted_scores.append(score * weight)
    
    # Calculate overall TMS
    TMS = sum(weighted_scores) * 100  # Convert to percentage
    
    return TMS, parameter_scores

print("✅ TMS calculator ready!")
print("\n🧮 The Tissue Maturation Score integrates:")
print("   • Multiple parameters (7 per tissue type)")
print("   • Weighted importance factors")
print("   • Normalized to 0-100% scale")

---

## 📊 **SECTION 4: Interactive Maturation Dashboard**

Now let's create the **comprehensive dashboard** to monitor tissue maturation!

In [ ]:
def plot_maturation_dashboard(tissue_type, culture_condition, time_days, day_of_interest):
    """
    Create comprehensive maturation monitoring dashboard
    
    Panels:
    1. Overall TMS time course
    2. Individual parameter trajectories
    3. Radar plot of current status
    4. Parameter contribution bar chart
    5. Key metrics summary table
    6. Recommendations
    """
    
    # Generate data
    data = generate_maturation_data(tissue_type, culture_condition, time_days)
    params = TISSUE_DATABASE[tissue_type]['parameters']
    
    # Calculate TMS over time
    TMS_over_time = []
    for idx in range(len(data)):
        current_vals = data.iloc[idx].to_dict()
        tms, _ = calculate_TMS(tissue_type, current_vals)
        TMS_over_time.append(tms)
    
    # Get current day data
    day_idx = int(day_of_interest / time_days * (len(data) - 1))
    current_values = data.iloc[day_idx].to_dict()
    current_TMS, param_scores = calculate_TMS(tissue_type, current_values)
    
    # Create figure with subplots
    fig = plt.figure(figsize=(18, 12))
    gs = fig.add_gridspec(3, 3, hspace=0.35, wspace=0.35)
    
    # ========================================================================
    # PANEL 1: Overall TMS Time Course (Top Left, Large)
    # ========================================================================
    ax1 = fig.add_subplot(gs[0, :])
    ax1.plot(data['Time'], TMS_over_time, 'b-', linewidth=3, label='TMS')
    ax1.axvline(day_of_interest, color='red', linestyle='--', linewidth=2, alpha=0.7, label=f'Day {day_of_interest}')
    ax1.axhline(80, color='green', linestyle=':', linewidth=2, alpha=0.5, label='Maturation Threshold (80%)')
    ax1.fill_between(data['Time'], 0, TMS_over_time, alpha=0.2)
    
    ax1.set_xlabel('Culture Time (days)', fontsize=13, fontweight='bold')
    ax1.set_ylabel('Tissue Maturation Score (%)', fontsize=13, fontweight='bold')
    ax1.set_title(f'📈 Overall Tissue Maturation Progress\n{tissue_type} - {culture_condition} Conditions', 
                  fontsize=15, fontweight='bold', pad=15)
    ax1.grid(True, alpha=0.3)
    ax1.legend(fontsize=11, loc='lower right')
    ax1.set_ylim(0, 105)
    
    # Add current TMS annotation
    ax1.scatter([day_of_interest], [current_TMS], s=200, c='red', zorder=5, edgecolors='black', linewidth=2)
    ax1.annotate(f'{current_TMS:.1f}%', 
                xy=(day_of_interest, current_TMS), 
                xytext=(10, 20), textcoords='offset points',
                fontsize=12, fontweight='bold',
                bbox=dict(boxstyle='round,pad=0.5', facecolor='yellow', alpha=0.7),
                arrowprops=dict(arrowstyle='->', connectionstyle='arc3,rad=0', lw=2))
    
    # ========================================================================
    # PANEL 2: Individual Parameter Trajectories (Middle Row)
    # ========================================================================
    ax2 = fig.add_subplot(gs[1, :])
    
    # Plot each parameter (normalized to 0-100%)
    colors = plt.cm.tab10(np.linspace(0, 1, len(params)))
    
    for idx, (param_name, color) in enumerate(zip(params.keys(), colors)):
        baseline = params[param_name]['baseline']
        target = params[param_name]['target']
        
        # Normalize to percentage
        normalized = (data[param_name] - baseline) / (target - baseline) * 100
        normalized = np.clip(normalized, 0, 100)
        
        ax2.plot(data['Time'], normalized, '-', linewidth=2.5, color=color, 
                label=param_name.replace('_', ' '), alpha=0.8)
    
    ax2.axvline(day_of_interest, color='red', linestyle='--', linewidth=2, alpha=0.5)
    ax2.axhline(80, color='green', linestyle=':', linewidth=1.5, alpha=0.3)
    
    ax2.set_xlabel('Culture Time (days)', fontsize=13, fontweight='bold')
    ax2.set_ylabel('Parameter Maturation (%)', fontsize=13, fontweight='bold')
    ax2.set_title('📊 Individual Parameter Development', fontsize=14, fontweight='bold', pad=10)
    ax2.legend(fontsize=9, loc='upper left', ncol=2, framealpha=0.9)
    ax2.grid(True, alpha=0.3)
    ax2.set_ylim(-5, 105)
    
    # ========================================================================
    # PANEL 3: Radar Plot (Bottom Left)
    # ========================================================================
    ax3 = fig.add_subplot(gs[2, 0], projection='polar')
    
    # Prepare radar data
    categories = list(params.keys())
    values = [param_scores[cat] for cat in categories]
    
    # Number of variables
    N = len(categories)
    angles = np.linspace(0, 2 * np.pi, N, endpoint=False).tolist()
    values += values[:1]  # Complete the circle
    angles += angles[:1]
    
    # Plot
    ax3.plot(angles, values, 'o-', linewidth=2, color='blue', label='Current')
    ax3.fill(angles, values, alpha=0.25, color='blue')
    ax3.plot(angles, [80]*len(angles), ':', linewidth=2, color='green', alpha=0.5, label='Target (80%)')
    
    # Customize
    ax3.set_xticks(angles[:-1])
    ax3.set_xticklabels([cat.replace('_', '\n') for cat in categories], fontsize=9)
    ax3.set_ylim(0, 100)
    ax3.set_yticks([25, 50, 75, 100])
    ax3.set_yticklabels(['25%', '50%', '75%', '100%'], fontsize=9)
    ax3.set_title(f'🎯 Current Status (Day {day_of_interest})', fontsize=12, fontweight='bold', pad=20)
    ax3.legend(loc='upper right', bbox_to_anchor=(1.3, 1.1), fontsize=9)
    ax3.grid(True)
    
    # ========================================================================
    # PANEL 4: Parameter Contributions (Bottom Middle)
    # ========================================================================
    ax4 = fig.add_subplot(gs[2, 1])
    
    # Calculate weighted contributions
    contributions = {}
    for param_name in params.keys():
        weight = params[param_name]['weight']
        score = param_scores[param_name]
        contributions[param_name] = weight * score
    
    # Sort by contribution
    sorted_params = sorted(contributions.items(), key=lambda x: x[1], reverse=True)
    param_names = [p[0].replace('_', '\n') for p in sorted_params]
    contrib_values = [p[1] for p in sorted_params]
    
    # Color by performance
    colors_bar = ['green' if v > 16 else 'orange' if v > 10 else 'red' for v in contrib_values]
    
    bars = ax4.barh(param_names, contrib_values, color=colors_bar, alpha=0.7, edgecolor='black')
    ax4.set_xlabel('Weighted Contribution to TMS', fontsize=11, fontweight='bold')
    ax4.set_title('⚖️ Parameter Contributions', fontsize=12, fontweight='bold', pad=10)
    ax4.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for i, (bar, val) in enumerate(zip(bars, contrib_values)):
        ax4.text(val + 0.5, i, f'{val:.1f}', va='center', fontsize=9, fontweight='bold')
    
    # ========================================================================
    # PANEL 5: Key Metrics Summary (Bottom Right)
    # ========================================================================
    ax5 = fig.add_subplot(gs[2, 2])
    ax5.axis('off')
    
    # Create summary text
    summary_text = f"""📋 KEY METRICS SUMMARY
━━━━━━━━━━━━━━━━━━━━━━━━━━━━

🔬 Tissue Type: {tissue_type}
⚗️ Culture Condition: {culture_condition}
📅 Culture Day: {day_of_interest}

━━━━━━━━━━━━━━━━━━━━━━━━━━━━

📊 OVERALL TMS: {current_TMS:.1f}%

━━━━━━━━━━━━━━━━━━━━━━━━━━━━

TOP PERFORMERS:
"""
    
    # Add top 3 parameters
    sorted_scores = sorted(param_scores.items(), key=lambda x: x[1], reverse=True)
    for i, (param, score) in enumerate(sorted_scores[:3], 1):
        summary_text += f"\n{i}. {param.replace('_', ' ')}: {score:.1f}%"
    
    summary_text += "\n\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n\nNEEDS IMPROVEMENT:\n"
    
    # Add bottom 3 parameters
    for i, (param, score) in enumerate(sorted_scores[-3:][::-1], 1):
        summary_text += f"\n{i}. {param.replace('_', ' ')}: {score:.1f}%"
    
    # Maturation status
    if current_TMS >= 80:
        status = "✅ MATURE"
        status_color = 'green'
    elif current_TMS >= 60:
        status = "🟡 MATURING"
        status_color = 'orange'
    else:
        status = "🔴 IMMATURE"
        status_color = 'red'
    
    summary_text += f"\n\n━━━━━━━━━━━━━━━━━━━━━━━━━━━━\n\nSTATUS: {status}"
    
    ax5.text(0.05, 0.95, summary_text, transform=ax5.transAxes,
            fontsize=10, verticalalignment='top', fontfamily='monospace',
            bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))
    
    plt.suptitle(f'🔬 TISSUE MATURATION MONITORING DASHBOARD 🔬', 
                fontsize=18, fontweight='bold', y=0.98)
    
    plt.tight_layout()
    plt.show()
    
    # ========================================================================
    # Print Recommendations
    # ========================================================================
    print("\n" + "="*80)
    print("💡 OPTIMIZATION RECOMMENDATIONS")
    print("="*80)
    
    # Find weakest parameters
    weak_params = [p for p, s in param_scores.items() if s < 50]
    
    if not weak_params:
        print("\n✅ Excellent progress! All parameters above 50% maturation.")
    else:
        print(f"\n⚠️ {len(weak_params)} parameter(s) below 50% maturation:")
        for param in weak_params:
            print(f"   • {param.replace('_', ' ')}: {param_scores[param]:.1f}%")
    
    # Condition-specific recommendations
    if culture_condition == 'Suboptimal':
        print("\n🔧 Consider optimizing culture conditions:")
        print("   • Increase medium exchange frequency")
        print("   • Add growth factors or biochemical cues")
        print("   • Apply mechanical stimulation")
        print("   • Check oxygen tension")
    
    if current_TMS < 60 and day_of_interest > time_days * 0.7:
        print("\n⏰ Maturation slower than expected for this timepoint.")
        print("   • Review culture protocol")
        print("   • Consider extending culture time")
        print("   • Verify cell source and quality")
    
    if current_TMS >= 80:
        print("\n🎉 Tissue has reached maturation threshold!")
        print("   • Consider harvesting for implantation/testing")
        print("   • Document final characterization")
        print("   • Archive successful protocol parameters")
    
    print("\n" + "="*80)

print("✅ Interactive dashboard ready!")
print("\n🎨 Dashboard includes:")
print("   1️⃣ Overall TMS trajectory")
print("   2️⃣ Individual parameter development")
print("   3️⃣ Current status radar plot")
print("   4️⃣ Parameter contribution analysis")
print("   5️⃣ Key metrics summary")
print("   6️⃣ Optimization recommendations")

---

## 🎮 **INTERACTIVE WIDGET: Explore the Dashboard!**

Use the controls below to explore different scenarios:

In [ ]:
# Create interactive widget
interact(plot_maturation_dashboard,
         tissue_type=widgets.Dropdown(
             options=['Cartilage', 'Bone', 'Cardiac', 'Vascular'],
             value='Cartilage',
             description='Tissue Type:',
             style={'description_width': '120px'}),
         
         culture_condition=widgets.Dropdown(
             options=['Optimal', 'Standard', 'Suboptimal'],
             value='Standard',
             description='Culture Conditions:',
             style={'description_width': '120px'}),
         
         time_days=widgets.IntSlider(
             value=28,
             min=14,
             max=56,
             step=7,
             description='Total Culture Time (days):',
             style={'description_width': '180px'},
             layout={'width': '500px'}),
         
         day_of_interest=widgets.IntSlider(
             value=21,
             min=0,
             max=56,
             step=1,
             description='Analysis Day:',
             style={'description_width': '120px'},
             layout={'width': '500px'})
);

---

## 🧮 **SECTION 5: Compare Multiple Culture Conditions**

Let's compare how different culture conditions affect final tissue maturation.

In [ ]:
def compare_conditions(tissue_type, time_days):
    """
    Compare all three culture conditions side-by-side
    """
    
    conditions = ['Optimal', 'Standard', 'Suboptimal']
    colors = ['green', 'orange', 'red']
    
    fig, axes = plt.subplots(2, 2, figsize=(16, 12))
    
    # Generate data for all conditions
    all_data = {}
    all_TMS = {}
    
    for cond in conditions:
        data = generate_maturation_data(tissue_type, cond, time_days)
        all_data[cond] = data
        
        # Calculate TMS over time
        TMS_over_time = []
        for idx in range(len(data)):
            current_vals = data.iloc[idx].to_dict()
            tms, _ = calculate_TMS(tissue_type, current_vals)
            TMS_over_time.append(tms)
        all_TMS[cond] = TMS_over_time
    
    # ========================================================================
    # PANEL 1: TMS Comparison
    # ========================================================================
    ax1 = axes[0, 0]
    for cond, color in zip(conditions, colors):
        ax1.plot(all_data[cond]['Time'], all_TMS[cond], '-', linewidth=3, 
                color=color, label=cond, alpha=0.8)
    
    ax1.axhline(80, color='black', linestyle=':', linewidth=2, alpha=0.5, label='Target (80%)')
    ax1.set_xlabel('Culture Time (days)', fontsize=12, fontweight='bold')
    ax1.set_ylabel('Tissue Maturation Score (%)', fontsize=12, fontweight='bold')
    ax1.set_title('📈 TMS Comparison Across Conditions', fontsize=13, fontweight='bold')
    ax1.legend(fontsize=11, loc='lower right')
    ax1.grid(True, alpha=0.3)
    ax1.set_ylim(0, 105)
    
    # ========================================================================
    # PANEL 2: Final TMS Bar Chart
    # ========================================================================
    ax2 = axes[0, 1]
    final_TMS = [all_TMS[cond][-1] for cond in conditions]
    bars = ax2.bar(conditions, final_TMS, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax2.axhline(80, color='black', linestyle=':', linewidth=2, alpha=0.5)
    ax2.set_ylabel('Final TMS (%)', fontsize=12, fontweight='bold')
    ax2.set_title(f'🎯 Final Maturation (Day {time_days})', fontsize=13, fontweight='bold')
    ax2.set_ylim(0, 105)
    ax2.grid(axis='y', alpha=0.3)
    
    # Add value labels
    for bar, val in zip(bars, final_TMS):
        ax2.text(bar.get_x() + bar.get_width()/2, val + 2, f'{val:.1f}%',
                ha='center', va='bottom', fontsize=11, fontweight='bold')
    
    # ========================================================================
    # PANEL 3: Parameter Comparison Heatmap
    # ========================================================================
    ax3 = axes[1, 0]
    
    # Get final parameter scores for each condition
    params = TISSUE_DATABASE[tissue_type]['parameters']
    param_names = list(params.keys())
    
    score_matrix = []
    for cond in conditions:
        final_vals = all_data[cond].iloc[-1].to_dict()
        _, param_scores = calculate_TMS(tissue_type, final_vals)
        score_matrix.append([param_scores[p] for p in param_names])
    
    im = ax3.imshow(score_matrix, cmap='RdYlGn', aspect='auto', vmin=0, vmax=100)
    
    # Set ticks
    ax3.set_xticks(np.arange(len(param_names)))
    ax3.set_yticks(np.arange(len(conditions)))
    ax3.set_xticklabels([p.replace('_', '\n') for p in param_names], fontsize=9)
    ax3.set_yticklabels(conditions, fontsize=11)
    
    # Add values
    for i in range(len(conditions)):
        for j in range(len(param_names)):
            text = ax3.text(j, i, f'{score_matrix[i][j]:.0f}',
                          ha="center", va="center", color="black", fontsize=9, fontweight='bold')
    
    ax3.set_title('🔥 Parameter Heatmap (Final Values)', fontsize=13, fontweight='bold')
    plt.colorbar(im, ax=ax3, label='Maturation (%)')
    
    # ========================================================================
    # PANEL 4: Time to 80% TMS
    # ========================================================================
    ax4 = axes[1, 1]
    
    time_to_threshold = []
    for cond in conditions:
        tms_values = np.array(all_TMS[cond])
        time_points = all_data[cond]['Time'].values
        
        # Find when TMS reaches 80%
        idx = np.where(tms_values >= 80)[0]
        if len(idx) > 0:
            time_to_80 = time_points[idx[0]]
        else:
            time_to_80 = time_days  # Didn't reach threshold
        
        time_to_threshold.append(time_to_80)
    
    bars = ax4.barh(conditions, time_to_threshold, color=colors, alpha=0.7, edgecolor='black', linewidth=2)
    ax4.set_xlabel('Days to Reach 80% TMS', fontsize=12, fontweight='bold')
    ax4.set_title('⏱️ Maturation Kinetics', fontsize=13, fontweight='bold')
    ax4.grid(axis='x', alpha=0.3)
    
    # Add value labels
    for bar, val in zip(bars, time_to_threshold):
        if val < time_days:
            label = f'{val:.1f} days'
        else:
            label = 'Not reached'
        ax4.text(val + 1, bar.get_y() + bar.get_height()/2, label,
                ha='left', va='center', fontsize=10, fontweight='bold')
    
    plt.suptitle(f'🔬 Multi-Condition Comparison: {tissue_type} Tissue', 
                fontsize=16, fontweight='bold', y=0.995)
    
    plt.tight_layout()
    plt.show()
    
    # Print summary
    print("\n" + "="*80)
    print("📊 CONDITION COMPARISON SUMMARY")
    print("="*80)
    
    for cond, final, time_80 in zip(conditions, final_TMS, time_to_threshold):
        print(f"\n{cond} Conditions:")
        print(f"   • Final TMS (Day {time_days}): {final:.1f}%")
        if time_80 < time_days:
            print(f"   • Time to 80% TMS: {time_80:.1f} days")
        else:
            print(f"   • Time to 80% TMS: Not reached")
    
    # Recommendation
    best_idx = np.argmax(final_TMS)
    best_condition = conditions[best_idx]
    
    print(f"\n{'='*80}")
    print(f"✅ RECOMMENDATION: Use {best_condition} conditions for optimal tissue maturation.")
    print(f"   Final TMS: {final_TMS[best_idx]:.1f}%")
    print(f"   Time to maturation: {time_to_threshold[best_idx]:.1f} days")
    print("="*80 + "\n")

print("✅ Condition comparison function ready!")

---

## 🎮 **INTERACTIVE WIDGET: Compare Conditions**

In [ ]:
interact(compare_conditions,
         tissue_type=widgets.Dropdown(
             options=['Cartilage', 'Bone', 'Cardiac', 'Vascular'],
             value='Cartilage',
             description='Tissue Type:',
             style={'description_width': '120px'}),
         
         time_days=widgets.IntSlider(
             value=28,
             min=14,
             max=56,
             step=7,
             description='Culture Time (days):',
             style={'description_width': '150px'},
             layout={'width': '500px'})
);

---

## ❓ **EXERCISE QUESTIONS**

Use the interactive dashboard to answer these questions:

### **Question 1: Cartilage Maturation**
Set: `Tissue Type = Cartilage`, `Culture Condition = Standard`, `Time = 28 days`, `Analysis Day = 21`

**a)** What is the overall TMS at day 21?  
**b)** Which parameter shows the BEST maturation?  
**c)** Which parameter needs the most improvement?  
**d)** Has the tissue reached the 80% maturation threshold by day 28?

---

### **Question 2: Effect of Culture Conditions**
Compare the three culture conditions for `Bone` tissue at `42 days`.

**a)** What is the final TMS for each condition?  
**b)** How many days does it take to reach 80% TMS under Optimal conditions?  
**c)** Which parameter shows the BIGGEST difference between Optimal and Suboptimal?  
**d)** If you could only improve ONE parameter in Suboptimal conditions, which should it be?

---

### **Question 3: Cardiac Tissue Challenge**
Set: `Tissue Type = Cardiac`, `Culture Condition = Suboptimal`, `Time = 28 days`

**a)** What is the final TMS?  
**b)** Is this tissue ready for testing/implantation (≥80% TMS)?  
**c)** Which 2 parameters contribute MOST to the overall TMS?  
**d)** Based on the recommendations, what changes would you make to the protocol?

---

### **Question 4: Vascular Graft Development**
Set: `Tissue Type = Vascular`, `Culture Condition = Optimal`, `Time = 35 days`, `Analysis Day = 28`

**a)** What is the TMS at day 28?  
**b)** Which mechanical parameter (Burst Pressure or Compliance) is more mature?  
**c)** Does the tissue continue maturing significantly after day 28?  
**d)** Would you recommend extending culture time? Why or why not?

---

### **Question 5: Multi-Parameter Analysis**
For `Cartilage` tissue under `Standard` conditions at `Day 21`:

**a)** Rank the parameters from best to worst maturation  
**b)** Calculate the weighted contribution of GAG_Content to the overall TMS  
(Hint: Contribution = Weight × Score)  
**c)** If you could double the maturation rate of ONE parameter, which would increase TMS the most?  
**d)** Why is Youngs_Modulus weighted more heavily than Cell_Viability?

---

### **Question 6: Culture Duration Optimization**
For `Bone` tissue under `Optimal` conditions:

**a)** At what day does TMS first exceed 80%?  
**b)** What is the TMS growth rate (% per day) between days 14-21?  
**c)** Compare the growth rate early (days 0-14) vs late (days 28-42). What do you observe?  
**d)** From an economic perspective, what is the optimal harvest time?

---

### **Question 7: Design Your Own Tissue**

Imagine you're engineering a **new tissue type** not in our database.

**a)** Choose 7 parameters you would monitor  
**b)** Assign weight factors (must sum to 1.0)  
**c)** Explain your reasoning for the weights  
**d)** What culture time would you predict is needed to reach 80% TMS?

---

## 💭 **Discussion Questions**

1. **Parameter Weighting:**
   - Why do different tissue types have different parameter weights?
   - Should mechanical properties always be weighted heavily?
   - How would clinical application affect weight assignments?

2. **Maturation Kinetics:**
   - Why does tissue maturation follow a sigmoidal curve?
   - What biological processes limit maturation speed?
   - Can maturation be accelerated without compromising quality?

3. **Multi-Parameter Integration:**
   - Is a single TMS score sufficient for quality control?
   - What are the risks of oversimplifying multiple parameters?
   - Should certain parameters be "gate-keepers" (minimum thresholds)?

4. **Culture Optimization:**
   - How do you identify the rate-limiting parameter?
   - What interventions improve specific parameters?
   - Is it better to extend time or improve conditions?

5. **Quality Standards:**
   - Who decides what constitutes "mature" tissue?
   - Should standards differ for research vs clinical applications?
   - How do you validate that TMS predicts in vivo performance?

6. **Real-World Implementation:**
   - How frequently should tissues be assessed?
   - Which measurements are non-destructive?
   - How do you handle batch-to-batch variability?

---

## 📚 **Key Takeaways**

✅ **Multi-parameter monitoring** is essential for comprehensive tissue assessment

✅ **Tissue Maturation Score (TMS)** integrates diverse metrics into a single actionable score:
$$\text{TMS} = \sum_{i=1}^{n} w_i \cdot \frac{P_i - P_{i,\text{baseline}}}{P_{i,\text{target}} - P_{i,\text{baseline}}}$$

✅ **Parameter weighting** reflects functional importance for specific tissue applications

✅ **Maturation kinetics** typically follow **sigmoidal curves** with three phases:
- **Lag phase**: Initial adaptation (days 0-7)
- **Growth phase**: Rapid maturation (days 7-21)
- **Plateau phase**: Approaching native tissue (days 21+)

✅ **Culture condition optimization** dramatically affects:
- Final maturation level (50-95% TMS)
- Time to reach targets (14-42+ days)
- Parameter balance (some improve faster than others)

✅ **Critical monitoring parameters** by category:

| Category | Examples | Importance |
|----------|----------|------------|
| **Cellular** | Viability, proliferation | Foundation |
| **ECM** | Collagen, GAG, elastin | Structure |
| **Mechanical** | Stiffness, strength | Function |
| **Metabolic** | Glucose, lactate, O₂ | Health |
| **Molecular** | Gene expression | Differentiation |

✅ **Optimal culture strategy** balances:
- **Time**: Longer ≠ always better (plateau effects)
- **Cost**: Medium, labor, facility expenses
- **Quality**: Achieving target TMS threshold
- **Functionality**: Meeting application requirements

✅ **Data-driven decisions** enable:
- Protocol refinement based on weak parameters
- Harvest timing optimization
- Quality control standardization
- Regulatory documentation

---

## 🔗 **Connection to Course Content**

This exercise integrates concepts from multiple chapters:

- **Chapter 5.3:** Mass transport (nutrient delivery affects maturation)
- **Chapter 5.4:** Mechanical stimulation (improves mechanical properties)
- **Chapter 5.5:** Bioreactor systems (enabling optimal conditions)
- **Chapter 5.6:** Scale-up (maintaining quality at larger scales)

**Related exercises:**
- **Exercise 1:** Bioreactor selection impacts maturation rates
- **Exercise 2:** Oxygen transfer affects metabolic parameters
- **Exercise 3:** Mechanical stimulation enhances stiffness
- **Exercise 5:** Perfusion improves ECM deposition
- **Exercise 6:** Scale-up economics inform harvest timing

---

## 🎯 **Learning Outcomes Achieved**

After completing this exercise, you can:

✅ Calculate composite maturation scores from multiple biomarkers  
✅ Analyze time-course tissue development data  
✅ Compare culture conditions using multivariate analysis  
✅ Identify rate-limiting parameters in tissue maturation  
✅ Make evidence-based decisions for protocol optimization  
✅ Design comprehensive monitoring strategies for new tissue types  
✅ Integrate cellular, biochemical, mechanical, and molecular parameters  

---

## 🚀 **Next Steps**

To deepen your understanding:

1. **Read:** Chapter 5.7 on tissue maturation assessment techniques
2. **Explore:** Real tissue engineering publications tracking multiple parameters
3. **Design:** A monitoring dashboard for your own research project
4. **Consider:** How machine learning could predict maturation trajectories
5. **Investigate:** Non-destructive monitoring technologies (imaging, spectroscopy)

---

*🎓 Developed for Master's-level Bioengineering Students*  
*Vrije Universiteit Brussel - Biofabrication Course 2025*  
*Exercise 7: Tissue Maturation Monitoring Dashboard*